In [1]:
import pandas as pd
import time
from sklearn.preprocessing import MultiLabelBinarizer

#### **Variables & Description**

|  variable_name | type |                    desc                    |
|:--------------:|:----:|:------------------------------------------:|
| proj           | df   | main DataFrame                             |
| services       | list | list of all subscription services          |
| all_gen        | list | list of all available genres               |
| countries_list | list | list of all countries                      |
| service_pipe   | df   | proj joined with services one hot encoding |


In [2]:
#initial set-up
proj = pd.read_csv('project.csv', names=['ShowName', 'Links', 'Imdb', 'Reelgood', 'Genres', 'MaturityRating', 'RunningDate', 'Seasons', 'ReturningDate', 'Service', 'Country'], header=0)

# for some fucking reason... pag ni iterate mo what looks like a god damn list... turns out.. it's not
# for i in proj.Country: #Same for Genres and Services
#     for j in i:
#         time.sleep(1)
#         print(j)

services = ['Netflix', 'Hulu', 'Prime Video', 'Disney+', 'HBO', 'HBO MAX', 
            'Peacock', 'Apple TV+', 'fuboTV', 'Showtime', 'Starz', 
            'CBS All Access','Epix', 'Crunchyroll', 'Funimation', 'AMC Premiere', 
            'Kanopy', 'Hoopla', 'The Criterion Channel', 'BritBox', 'DC Universe', 
            'Mubi', 'Cinemax', 'Fandor', 'AcornTV', 'Hallmark Movies Now', 'BET+', 
            'Youtube Premium','IndieFlix']

all_gen = ['Action & Adventure', 'Animation', 'Anime', 'Children', 'Comedy', 'Crime', 
           'Documentary', 'Drama', 'Family', 'Fantasy', 'Food', 'Game Show','Home & Garden',
           'Horror','LGBTQ', 'Mystery', 'Reality', 'Science-Fiction', 'Sport', 'Stand-up & Talk', 'Travel']

countries_list = ['America','Argentina','Australia','Austria','Belgium','Brazil','British Indian Ocean Territory','Bulgaria','Canada',
                  'Chile','China','Colombia','Columbia','Croatia','Czechia','Denmark',
                  'Egypt','Finland','France','Germany','Hong Kong','Hungary',
                  'Iceland','India','Iraq','Ireland','Israel','Italy','Japan',
                  'Luxembourg','Malaysia','Mexico','Netherlands','New Zealand',
                  'Norway','Philippines','Poland','Puerto Rico','Russia','South Africa',
                  'South Korea','Spain','Sweden','Switzerland','Taiwan','Thailand','Turkey','United Kingdom']

def New_Services(serve):
    new_list=[]
    for service in services:
        if service in serve:
            new_list.append(service)
    return new_list

def New_Genres(more_gen):
    Genre = []
    for gen in all_gen:
        if gen in more_gen:
            Genre.append(gen)
    return Genre

def New_Countries(cntry):
    new_country = []
    for i in countries_list:
        if i in cntry:
            new_country.append(i)
    return new_country

proj['Service'] = proj['Service'].apply(New_Services)
proj['Genres'] = proj['Genres'].astype(str).apply(New_Genres)
proj['Country'] = proj['Country'].astype(str).apply(New_Countries)

proj.head()

,ShowName,Links,Imdb,Reelgood,Genres,MaturityRating,RunningDate,Seasons,ReturningDate,Service,Country
0,Breaking Bad,https://reelgood.com/show/breaking-bad-2008,9.5,100.0,"[Crime, Drama]",Rated: 18+ (TV-MA),2008 - 2013,5 Seasons,Series Ended,[Netflix],[America]
1,Game of Thrones,https://reelgood.com/show/game-of-thrones-2011,9.3,99.0,"[Action & Adventure, Drama, Fantasy, Science-F...",Rated: 18+ (TV-MA),2011 - 2019,8 Seasons,Series Ended,"[HBO, HBO MAX]",[America]
2,Rick and Morty,https://reelgood.com/show/rick-and-morty-2013,9.2,97.0,"[Action & Adventure, Animation, Comedy, Fantas...",Rated: 18+ (TV-MA),2013 - Present,4 Seasons,Returning: Date TBA,"[Hulu, HBO, HBO MAX, fuboTV, Hoopla]",[America]
3,Stranger Things,https://reelgood.com/show/stranger-things-2016,8.8,96.0,"[Action & Adventure, Drama, Fantasy, Horror, M...",Rated: 14+ (TV-14),2016 - Present,3 Seasons,Returning: Date TBA,[Netflix],[America]
4,Dark,https://reelgood.com/show/dark-2017,8.8,95.0,"[Crime, Drama, Fantasy, Mystery, Science-Fiction]",Rated: 14+ (TV-14),2017 - 2020,3 Seasons,Series Ended,[Netflix],[Germany]


- Running dates:
    - inspect how many years did the show ran and it's ratings
    - how many shows are still running.
        - ratings
        - genres
    - how many shows has ended.
        - ratings
        - genres
- Genres:
    - how many shows per genre: %
    - highest rated per genre
- Maturity rating:
    - how many shows per rating %
    - highest rated per rating
- Services:
    - shows available per Service
    - top shows per service
- Correlation:
    - ratings vs running time
    - ratings vs genres
    - ratings vs countries
    - ratings vs maturity ratings
    - does ratings affect the no of seasons

In [3]:
proj.columns 

Index(['ShowName', 'Links', 'Imdb', 'Reelgood', 'Genres', 'MaturityRating',
       'RunningDate', 'Seasons', 'ReturningDate', 'Service', 'Country'],
      dtype='object')

In [4]:
proj.shape

(9778, 11)

In [5]:
proj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9778 entries, 0 to 9777
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ShowName        9778 non-null   object 
 1   Links           9778 non-null   object 
 2   Imdb            8282 non-null   float64
 3   Reelgood        9294 non-null   float64
 4   Genres          9778 non-null   object 
 5   MaturityRating  6017 non-null   object 
 6   RunningDate     9778 non-null   object 
 7   Seasons         9778 non-null   object 
 8   ReturningDate   9778 non-null   object 
 9   Service         9778 non-null   object 
 10  Country         9778 non-null   object 
dtypes: float64(2), object(9)
memory usage: 840.4+ KB


In [6]:
proj.dtypes

ShowName           object
Links              object
Imdb              float64
Reelgood          float64
Genres             object
MaturityRating     object
RunningDate        object
Seasons            object
ReturningDate      object
Service            object
Country            object
dtype: object

In [7]:
proj.describe()

,Imdb,Reelgood
count,8282.000000,9294.000000
mean,7.042272,49.406606
std,1.116074,17.560583
min,1.000000,10.000000
25%,6.500000,39.000000
50%,7.200000,50.000000
75%,7.800000,61.000000
max,9.600000,100.000000


In [8]:
# Daming NaN Countries
proj.isna().sum()

ShowName             0
Links                0
Imdb              1496
Reelgood           484
Genres               0
MaturityRating    3761
RunningDate          0
Seasons              0
ReturningDate        0
Service              0
Country              0
dtype: int64

What are the top 10 shows on IMDB that are available on each streaming service?

In [9]:
# One Hot encoding for services
mlb = MultiLabelBinarizer()
service_pipe = proj.join(pd.DataFrame(mlb.fit_transform(proj.pop('Service')),
                          columns=mlb.classes_,
                          index=proj.index))

# mlb.classes_